In [2]:
import gquant
import abupy
import QUANTAXIS as QA
import gquant.jupyter_helper
gquant.jupyter_helper.init()

Expand macros in C:\Users\GuQiang\Anaconda3\envs\stock_ai\lib\site-packages\macropy\core\hquotes.py
Finding macros in 'macropy.core.hquotes'
Importing macros from 'macropy.core.quotes' into 'macropy.core.hquotes'
Expand macros in C:\Users\GuQiang\Anaconda3\envs\stock_ai\lib\site-packages\macropy\core\quotes.py
Finding macros in 'macropy.core.quotes'
Expand macros in C:\Users\GuQiang\Anaconda3\envs\stock_ai\lib\site-packages\macropy\core\failure.py
Finding macros in 'macropy.core.failure'
Importing macros from 'macropy.core.hquotes' into 'macropy.core.failure'
Expand macros in C:\Users\GuQiang\Anaconda3\envs\stock_ai\lib\site-packages\numba\core\pythonapi.py
Finding macros in 'numba.core.pythonapi'
Expand macros in C:\Users\GuQiang\Anaconda3\envs\stock_ai\lib\site-packages\numba\core\rewrites\__init__.py
Finding macros in 'numba.core.rewrites'
Expand macros in C:\Users\GuQiang\Anaconda3\envs\stock_ai\lib\site-packages\numba\core\rewrites\macros.py
Finding macros in 'numba.core.rewrites.

In [3]:
symbol_list=QA.QA_fetch_stock_list_adv()

_start='2014-01-01'
_end='2018-06-31'

In [4]:
class LocalDataAPI(abupy.MarketBu.ABuDataBase.StockBaseMarket,
                   abupy.MarketBu.ABuDataBase.SupportMixin):
    def _support_market(self):
        """声明数据源支持A股"""
        return [abupy.EMarketTargetType.E_MARKET_TARGET_CN]

    def kline(self, n_folds=2, start=None, end=None):
        df=QA.QA_fetch_stock_day_adv(self._symbol.symbol_code,start,end).to_qfq().data.copy()
        df['pre_close'] = df['close'].shift()
        df['date'] = df.index.get_level_values(0)
        df['date'] = df['date'].dt.strftime('%Y%m%d').apply(
            lambda x: abupy.ABuDateUtil.date_str_to_int(x))
        df.index = df.index.droplevel(1)
        df.index.name = 'dt'
        df['date_week'] = df.index.dayofweek
        if start and end:
            return df[start:end]
        elif start:
            return df[start:]
        elif end:
            return df[:end]
        return df

    def minute(self, n_fold=5, *args, **kwargs):
        """分钟k线接口"""
        raise NotImplementedError('QAApi minute NotImplementedError!')

In [5]:
abupy.env.g_market_target = abupy.env.EMarketTargetType.E_MARKET_TARGET_CN
# 所有任务数据强制网络更新
abupy.env.g_data_fetch_mode = abupy.env.EMarketDataFetchMode.E_DATA_FETCH_FORCE_NET
# 使用QUATAXIS本地数据作为数据源
abupy.env.g_private_data_source = LocalDataAPI

In [6]:
from abupy.IndicatorBu.ABuNDAtr import calc_atr
from QUANTAXIS.QAIndicator.talib_series import SMA
from QUANTAXIS.QAIndicator.talib_series import BBANDS

class KLManager(abupy.AbuKLManager):
    def _fetch_pick_time_kl_pd(self, target_symbol):
        data = abupy.MarketBu.ABuSymbolPd.make_kl_df(target_symbol,start=self.benchmark.start,end=self.benchmark.end)
        if not data.empty:
            high = data['high']
            low = data['low']
            pre_close = data['pre_close']
            data['ma']=SMA(data['close'],20)
            data['atr']=calc_atr(high, low, pre_close, 20)
            data['up']=data['ma']+data['atr']
            data['low']=data['ma']-data['atr']
            data['atr14'] = calc_atr(high, low, pre_close, 14)
            data['atr21'] = calc_atr(high, low, pre_close, 21)
            # 收盘价上穿20日移动均线+20日ATR
            data['buy']=QA.QAIndicator.CROSS(data['close'],data['up'])
            # 收盘价下穿20日移动均线-20日ATR
            data['sell']=QA.QAIndicator.CROSS(data['low'],data['close'])
            result = data[self.benchmark.start:self.benchmark.end]
            result.name = data.name
        else:
            return data
#         logging.info(result)
        return result

## 仓位控制方法

In [7]:
class Position1(abupy.AbuPositionBase):
    """1手"""
    def _init_self(self, **kwargs):
        """子类仓位管理针对可扩展参数的初始化"""
        pass

    def fit_position(self, factor_object):
        return 100

## 回测方法

In [8]:
def backtest(symbol,buy_factors,sell_factors,init_cash=10000,**kwargs):
    start = kwargs.pop('start',_start)
    end = kwargs.pop('end',_end)

    benchmark = abupy.AbuBenchmark(symbol, start, end)
    capital = abupy.AbuCapital(init_cash, benchmark)
    klmanager=kwargs.pop('klmanager',KLManager)
    orders_pd, action_pd, _ = abupy.ABuPickTimeExecute.do_symbols_with_same_factors(
        [symbol],
        benchmark,
        buy_factors,
        sell_factors,
        capital,
        kl_pd_manager=klmanager(benchmark, capital),
        show=False)
    m = gquant.Metrics(orders_pd, action_pd, capital, benchmark)
    m.fit_metrics()
    if hasattr(m, 'act_sell') and not m.act_sell.empty:
        m.act_sell['profit_init']=m.act_sell['profit'] / m.capital.read_cash
        m.act_sell['profit_init_hunder'] = m.act_sell['profit_init'] * 100
        return m
    return None

## 买卖策略

### 买卖-ATR波幅

In [9]:
class BuyStrategy1(gquant.abupy.AbuFactorBuyTD, gquant.abupy.BuyCallMixin):
    def _init_self(self, **kwargs):
        self.factor_name = self.__class__.__name__

    def fit_day(self, today):
        if today['buy']:
            return self.buy_tomorrow()
        return None
    
class SellStrategy1(gquant.abupy.AbuFactorSellBase):
    def _init_self(self, **kwargs):
        self.factor_name = self.__class__.__name__

    def support_direction(self):
        """支持的方向，只支持正向"""
        return [abupy.ESupportDirection.DIRECTION_CAll.value]

    def fit_day(self, today, orders):
        if today['sell']:
            for order in orders:
                self.sell_tomorrow(order)

### 卖-最大持仓天数（20天）

In [10]:
_max_hold_days=20

class SellStrategy2(abupy.AbuFactorSellBase):
    """n日卖出策略。当N日后股价小于等于买入价时卖出。"""

    def _init_self(self, **kwargs):
        """kwargs中可以包含: 参数sell_n：代表买入后持有的天数，默认20天"""
        self.sell_n = kwargs.pop('sell_n', _max_hold_days)
        self.sell_type_extra = '{}:sell_n={}'.format(self.__class__.__name__, self.sell_n)

    def support_direction(self):
        """因子支持两个方向"""
        return [abupy.ESupportDirection.DIRECTION_CAll.value]


    def fit_day(self, today, orders):
        for order in orders:
            # 将单子的持有天数进行增加
            order.keep_days += 1
            if order.keep_days >= self.sell_n and order.buy_price>=today.close:
                self.sell_tomorrow(order)

### 买-当日价格超过购买价+2倍购买日ATR

In [11]:
class BuyStrategy2(gquant.abupy.AbuFactorBuyTD, gquant.abupy.BuyCallMixin):
    def _init_self(self, **kwargs):
        self.factor_name = self.__class__.__name__

    def fit_day(self, today):
        if self.orders:
            o=self.orders[-1]
            if o.sell_date is None:
                buy=self.kl_pd[self.kl_pd['date']==o.buy_date]#最后一次购买当日的信息
                atr=buy.iloc[0]['atr']#最后一次购买当日的atr数据
                if today.close>today.up and today.close>o.buy_price+2*atr:
                    return self.buy_tomorrow()

# 测试

In [12]:
ts = {
    '原始ATR波幅': {
        # 买入因子
        'buy_factors': [
            {
                'class': BuyStrategy1,
                # 仓位管理
                'position': {
                    'class': Position1
                }
            },
        ],
        # 卖出因子
        'sell_factors': [{
            'class': SellStrategy1,
        }]
    },
#     '原始ATR波幅+最多持仓天数': {
#         # 买入因子
#         'buy_factors': [
#             {
#                 'class': BuyStrategy1,
#                 # 仓位管理
#                 'position': {
#                     'class': Position1
#                 }
#             },
#         ],
#         # 卖出因子
#         'sell_factors': [{
#             'class': SellStrategy1,
#         }, {
#             'class': SellStrategy2,
#         }]
#     },
#     '原始ATR波幅+两倍ATR加仓': {
#         # 买入因子
#         'buy_factors': [
#             {
#                 'class': BuyStrategy1,
#                 # 仓位管理
#                 'position': {
#                     'class': Position1
#                 }
#             },
#             {
#                 'class': BuyStrategy2,
#                 # 仓位管理
#                 'position': {
#                     'class': Position1
#                 }
#             }
#         ],
#         # 卖出因子
#         'sell_factors': [{
#             'class': SellStrategy1,
#         }]
#     },
#     '原始ATR波幅+最多持仓天数+两倍ATR加仓': {
#         # 买入因子
#         'buy_factors': [
#             {
#                 'class': BuyStrategy1,
#                 # 仓位管理
#                 'position': {
#                     'class': Position1
#                 }
#             },
#             {
#                 'class': BuyStrategy2,
#                 # 仓位管理
#                 'position': {
#                     'class': Position1
#                 }
#             }
#         ],
#         # 卖出因子
#         'sell_factors': [{
#             'class': SellStrategy1,
#         }, {
#             'class': SellStrategy2,
#         }]
#     },
}

In [ ]:
from tqdm import tqdm_notebook
from IPython.display import clear_output
import logging

def getreport(bts):
    report=[]
    for symbol,metrics in bts.items():
        for name,m in metrics.items():
            if m is None:
                continue
            p=m.profit_series(name=name)
            p=gquant.pd.to_numeric(p)
            p['code']=symbol
            report.append(p)
    if not report:
        return gquant.pd.DataFrame({'R':[]})
    report=gquant.pd.concat(report, axis=1).T
    for col in report.columns:
        if col=='code':
            continue
        report[col]=gquant.pd.to_numeric(report[col])
    return report

bts={}

for symbol in tqdm_notebook(symbol_list.code):
#     if not symbol.startswith('30'):
#         continue
# for symbol in ['300005','300378','600036','601138','601099','600028']:
    bts[symbol]={}
    for k,v in ts.items():
        try:
            m=backtest(symbol,v['buy_factors'],v['sell_factors'],start='2014-01-01',end='2019-12-31')
            if m is None:
                continue
            bts[symbol][k]=m
        except:
#             raise
            pass
    rp=getreport(bts)
#     clear_output(wait=True)
    print(rp[['code','R','最终价值','最大回撤']].sort_values('R',ascending=False).head())
#     if symbol>='000005':
#         break
            
clear_output(wait=True)

In [27]:
report=getreport(bts)
report[(report['交易次数']>30)&(report['R']>3)].sort_values('R',ascending=False)

,盈亏总额,最终价值,交易次数,盈利次数,亏损次数,盈利比率,每笔交易平均盈亏额,盈利交易平均盈利额,亏损交易平均亏损额,R,最大回撤,买入平均花费,code
原始ATR波幅,34469.6332,43324.255,61.0,36.0,25.0,0.5902,545.0958,957.4898,-48.7517,11.18,-0.2044,833.5446,600021
原始ATR波幅,23818.2292,32735.092,63.0,29.0,34.0,0.4603,360.6365,821.3182,-32.2979,11.17,-0.2501,605.3607,601727
原始ATR波幅,42447.8279,50825.044,42.0,24.0,18.0,0.5714,971.7299,1768.6595,-90.8430,10.70,-0.2263,1621.8817,300168
原始ATR波幅,14050.8704,23436.451,42.0,20.0,22.0,0.4762,318.8703,702.5435,-29.9235,10.66,-0.2492,790.9668,600986
原始ATR波幅,8542.6824,18030.049,31.0,10.0,21.0,0.3226,257.9509,854.2682,-26.0098,9.92,-0.1879,715.2275,600654
...,...,...,...,...,...,...,...,...,...,...,...,...,...
原始ATR波幅,15509.6537,23313.531,56.0,28.0,28.0,0.5000,237.6829,553.9162,-78.5504,3.03,-0.1411,1328.7589,000815
原始ATR波幅,14173.6652,22407.629,51.0,29.0,22.0,0.5686,243.2106,488.7471,-80.4512,3.02,-0.2318,862.2419,600179
原始ATR波幅,11983.3323,22371.512,51.0,21.0,30.0,0.4118,196.6146,570.6349,-65.1996,3.02,-0.1621,1214.3373,002276
原始ATR波幅,7844.5361,16960.226,42.0,26.0,16.0,0.6190,165.8178,301.7129,-55.0119,3.01,-0.1720,1197.0503,300405


In [25]:
# report.groupby('code').agg({'R':['min', 'mean', 'max','median','std']})['R'].sort_values('mean',ascending=False)

# 蒙特卡洛模拟

采用完整数据，从中随机选出500~1500天的连续数据，使用[凯利公式对仓位进行管理](#使用凯利公式对仓位进行管理)中的买卖策略进行测试。

In [ ]:
_MC_SYMBOL='601099'

In [ ]:
import random
from QUANTAXIS.QAUtil.QADate import QA_util_datetime_to_strdate as dt2str

def generator(symbol):
    df=QA.QA_fetch_stock_day_adv(symbol,_start,_end).data
    """随机选择迭代器"""
    while True:
        size=random.randint(500,len(df)-10)
        start=random.randint(0,len(df)-size)
        end=start+size
        v= df.iloc[start:end].reset_index()
        yield dt2str(v.iloc[0]['date'].date()),dt2str(v.iloc[-1]['date'].date())
g=generator(_MC_SYMBOL)

In [ ]:
_buy_factors = [
    {
        'class': BuyStrategy1,
        # 仓位管理
        'position': {
            'class': Position1
        }
    },
    {
        'class': BuyStrategy2,
        # 仓位管理
        'position': {
            'class': Position1
        }
    }
]
# 卖出因子
_sell_factors = [{
    'class': SellStrategy1,
}, {
    'class': SellStrategy2,
}]


def do_mc_test(times):
    from tqdm import tqdm_notebook
    test = []
    for i in tqdm_notebook(range(times)):
        s, e = next(g)
        #         logging.info('LEN:{}:{}~{}'.format(len(test),s,e))
        mc = backtest(_MC_SYMBOL,_buy_factors, _sell_factors, start=s, end=e)
        if mc:
            test.append(mc)
    result = gquant.pd.concat([m.profit_series() for m in test], axis=1).T
    return result


def show_result(times, mcs):
    gquant.sns.distplot(mcs[mcs['交易次数'] >= 30]['R'])
    logging.info("测试{}次后交易次数>30的R乘数：\n{}".format(
        times, mcs[mcs['交易次数'] >= 30]['R'].describe()))

## 100

In [ ]:
# mcs100=do_mc_test(100)
# show_result(100,mcs100)

## 1000

In [ ]:
# mcs1000=do_mc_test(1000)
# show_result(1000,mcs1000)

## 10000

In [ ]:
# mcs10000=do_mc_test(10000)
# show_result(10000,mcs10000)